# ಸೆಮ್ಯಾಂಟಿಕ್ ಕರ್ಣಲ್

ಈ ಕೋಡ್ ಮಾದರಿಯಲ್ಲಿ, ನೀವು [ಸೆಮ್ಯಾಂಟಿಕ್ ಕರ್ಣಲ್](https://aka.ms/ai-agents-beginners/semantic-kernel) AI ಫ್ರೇಮ್ವರ್ಕ್ ಅನ್ನು ಬಳಸಿಕೊಂಡು ಒಂದು ಮೂಲ ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸುತ್ತೀರಿ.

ಈ ಮಾದರಿಯ ಉದ್ದೇಶವು ವಿವಿಧ ಏಜೆಂಟಿಕ್ ಮಾದರಿಗಳನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸುವಾಗ ನಾವು ನಂತರ ಬಳಸುವ ಹಂತಗಳನ್ನು ನಿಮಗೆ ತೋರಿಸುವುದಾಗಿದೆ.


## ಅಗತ್ಯವಿರುವ ಪೈಥಾನ್ ಪ್ಯಾಕೇಜ್‌ಗಳನ್ನು ಆಮದುಮಾಡಿ


In [ ]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## ಕ್ಲೈಂಟ್ ರಚನೆ

ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಾವು [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ಅನ್ನು LLM ಗೆ ಪ್ರವೇಶಿಸಲು ಬಳಸುತ್ತೇವೆ.

`ai_model_id` ಅನ್ನು `gpt-4o-mini` ಎಂದು ವ್ಯಾಖ್ಯಾನಿಸಲಾಗಿದೆ. GitHub Models ಮಾರುಕಟ್ಟೆಯಲ್ಲಿ ಲಭ್ಯವಿರುವ ಮತ್ತೊಂದು ಮಾದರಿಗೆ ಮಾದರಿಯನ್ನು ಬದಲಾಯಿಸಲು ಪ್ರಯತ್ನಿಸಿ ಮತ್ತು ವಿಭಿನ್ನ ಫಲಿತಾಂಶಗಳನ್ನು ನೋಡಿ.

GitHub Models ಗೆ `base_url` ಅನ್ನು ಬಳಸಲು ನಾವು ಬಳಸುವ `Azure Inference SDK` ಅನ್ನು ಬಳಸಲು, ನಾವು Semantic Kernel ನಲ್ಲಿ `OpenAIChatCompletion` ಕನೆಕ್ಟರ್ ಅನ್ನು ಬಳಸುತ್ತೇವೆ. Semantic Kernel ಅನ್ನು ಇತರ ಮಾದರಿ ಪೂರೈಕೆದಾರರಿಗಾಗಿ ಬಳಸಲು ಇತರ [ಲಭ್ಯವಿರುವ ಕನೆಕ್ಟರ್‌ಗಳು](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) ಕೂಡಾ ಇವೆ.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸುವುದು

ಕೆಳಗಿನ ಉದಾಹರಣೆಯಲ್ಲಿ ನಾವು `TravelAgent` ಎಂಬ ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸುತ್ತೇವೆ.

ಈ ಉದಾಹರಣೆಗೆ, ನಾವು ತುಂಬಾ ಸರಳವಾದ ಸೂಚನೆಗಳನ್ನು ಬಳಸುತ್ತಿದ್ದೇವೆ. ನೀವು ಈ ಸೂಚನೆಗಳನ್ನು ಬದಲಾಯಿಸಿ, ಏಜೆಂಟ್ ಹೇಗೆ ವಿಭಿನ್ನವಾಗಿ ಪ್ರತಿಕ್ರಿಯಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ನೋಡಬಹುದು.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ಏಜೆಂಟ್‌ಗಳನ್ನು ಚಲಾಯಿಸುವುದು

ಈಗ ನಾವು `ChatHistory` ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸಿ, ಅದಕ್ಕೆ `system_message` ಅನ್ನು ಸೇರಿಸುವ ಮೂಲಕ ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸಬಹುದು. ನಾವು ಮೊದಲು ವ್ಯಾಖ್ಯಾನಿಸಿದ `AGENT_INSTRUCTIONS` ಅನ್ನು ಇಲ್ಲಿ ಬಳಸುತ್ತೇವೆ.

ಇವುಗಳನ್ನು ವ್ಯಾಖ್ಯಾನಿಸಿದ ನಂತರ, ನಾವು `user_inputs` ಅನ್ನು ರಚಿಸುತ್ತೇವೆ, ಇದು ಬಳಕೆದಾರನು ಏಜೆಂಟ್‌ಗೆ ಕಳುಹಿಸುತ್ತಿರುವ ಸಂದೇಶವಾಗಿರುತ್ತದೆ. ಈ ಸಂದರ್ಭದಲ್ಲಿ, ನಾವು ಈ ಸಂದೇಶವನ್ನು `Plan me a sunny vacation` ಎಂದು ಹೊಂದಿಸಿದ್ದೇವೆ.

ಏಜೆಂಟ್ ಹೇಗೆ ವಿಭಿನ್ನವಾಗಿ ಪ್ರತಿಕ್ರಿಯಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ನೋಡಲು ಈ ಸಂದೇಶವನ್ನು ಬದಲಾಯಿಸಲು ಮುಕ್ತವಾಗಿರಿ.


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸಮೀಕ್ಷೆ**:  
ಈ ದಾಖಲೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಎಂಬ AI ಅನುವಾದ ಸೇವೆಯನ್ನು ಬಳಸಿಕೊಂಡು ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯನ್ನು ಸಾಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಮಂಜಸತೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಾಖಲೆ ಪ್ರಾಮಾಣಿಕ ಮೂಲವಾಗಿ ಪರಿಗಣಿಸಬೇಕು. ಪ್ರಮುಖ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪುಅರ್ಥಗಳು ಅಥವಾ ತಪ್ಪುಅನುವಾದಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
